<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-21 13:45:11
-------------------
qualified stocks: 93
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.39 L
Current:  1.38 C
-------------------
Today PnL: -18.86 K (-0.14%)
Current PnL: -17.12 L (-11.64%)
CY Booked + Current PnL: -8.39 L (-5.71%)
-------------------
Total profit:  3.76 L
Total loss:  -20.88 L
-------------------
Total Booked + Current PnL: 18.59 L (15.49%)
Total Booked PnL: 35.71 L (29.75%)
Curr Year Booked PnL: 8.73 L (6.32%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 76.23 L (55.16%)
Deployed:  1.20 C
Current:  1.38 C
CAGR/XIRR %: 9.0%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,HINDUNILVR,2413.81,2723.00,-5.27,H-LC,6.99,297741.0,27394.0,7235.0,-0.43,10.13,2.43,12.81,24.0,3.79,2.21,24.52,XY25,NTT,FMCG
36,HINDALCO,651.95,756.01,-18.25,H-LC,7.03,112824.0,8512.0,8135.0,0.61,8.16,7.21,15.96,9.0,1.05,0.84,26.44,X5K,ATH,METALS
4,APOLLOHOSP,7098.95,8285.00,-12.98,H-LC,7.30,173855.0,17678.0,8415.0,0.25,11.32,4.84,16.71,22.0,2.10,1.29,30.73,X40N,BTT,HEALTHCARE
68,SBIN,760.30,863.00,-11.34,M-LC,3.89,219128.0,17648.0,9576.0,-0.25,8.76,4.37,13.51,60.0,1.84,1.63,22.46,XY25,NTT,BANKS
55,MEDANTA,1087.93,1486.00,12.80,H-MC,14.36,161159.0,34959.0,11217.0,0.82,27.70,6.96,36.59,108.0,3.12,1.20,43.69,XY24,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-30.43,H-MC,7.37,178823.0,-985.0,113392.0,-0.05,-0.55,63.41,62.52,88.0,-0.01,1.33,33.35,XR,NTT,BANKS
58,PGHH,13388.00,18062.58,-30.68,H-MC,5.74,200640.0,-180.0,70304.0,-0.46,-0.09,35.04,34.92,80.0,-0.00,1.49,4.58,X40,ATH,FMCG
56,NATIONALUM,189.63,247.44,-49.46,H-MC,1.63,101479.0,975.0,29662.0,-0.39,0.97,29.23,30.49,79.0,0.03,0.75,33.69,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,ITC,411.90,452.0,-38.34,H-LC,3.06,147802.0,-2130.0,16731.0,-0.75,-1.42,11.32,9.74,4.0,-0.13,1.10,4.73,X40,NTT,FMCG
21,CIPLA,1492.70,1795.0,-19.44,H-LC,3.58,214610.0,10110.0,31312.0,1.32,4.94,14.59,20.25,10.0,0.32,1.59,14.11,X40N,BTT,PHARMA
53,LTIM,5564.16,7230.2,-5.20,H-LC,3.68,188352.0,-11958.0,71932.0,0.99,-5.97,38.19,29.94,16.0,-0.17,1.40,30.12,X200,ATH,IT
57,NESTLEIND,1134.30,1377.0,-10.13,H-LC,3.72,276588.0,11162.0,45637.0,-0.70,4.21,16.50,21.40,11.0,0.24,2.05,12.32,XY25,NTT,FMCG
5,ASIANPAINT,2961.56,4250.0,-10.45,H-LC,5.24,174780.0,-26606.0,114219.0,0.01,-13.21,65.35,43.51,27.0,-0.23,1.30,21.66,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-39.64,L-SC,23.30,88397.0,-25152.0,65202.0,15.77,-22.15,73.76,35.27,268.0,-0.39,0.66,116.35,XR,NTT,HOTELS
60,RAJESHEXPO,517.65,518.00,1735.86,L-SC,7.72,50106.0,-87071.0,87164.0,2.25,-63.47,173.96,0.07,267.0,-1.00,0.37,24.62,OX40N,NTT,JEWELLERY
75,SURYODAY,189.78,240.00,44.74,H-SC,5.34,103103.0,-46823.0,86493.0,1.95,-31.23,83.89,26.46,135.0,-0.54,0.76,32.08,XR,NTT,BANKS
66,SAMMAANCAP,170.35,326.00,-164.77,M-SC,7.55,74280.0,-27930.0,121322.0,1.93,-27.33,163.33,91.37,208.0,-0.23,0.55,20.58,XY25,NTT,FINANCE
71,SIEMENS,4406.90,7969.85,-0.04,H-LC,34.19,80245.0,-29927.0,119003.0,1.73,-27.16,148.30,80.85,15.0,-0.25,0.60,18.19,AR,ATH,ELECTRICAL


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,EASEMYTRIP,18.12,26.4,-5.18,M-SC,26.62,91237.0,-85578.0,166371.0,-3.21,-48.40,182.35,45.70,197.0,-0.51,0.68,6.13,XY24,NTT,TRAVEL
87,VAIBHAVGBL,350.21,521.0,42.70,H-SC,10.25,95159.0,-56832.0,130958.0,-2.33,-37.39,137.62,48.77,125.0,-0.43,0.71,10.76,XR,NTT,JEWELLERY
14,BERGEPAINT,561.33,680.0,-20.05,H-MC,9.03,215055.0,-12284.0,60344.0,-2.30,-5.40,28.06,21.14,106.0,-0.20,1.60,21.22,XY24,NTT,PAINTS
25,DABUR,505.20,735.0,-6.36,H-MC,3.80,203970.0,6942.0,82689.0,-2.24,3.52,40.54,45.49,102.0,0.08,1.51,17.02,XY24,BTT,FMCG
50,KANSAINER,299.63,340.0,-68.70,H-SC,2.94,215865.0,-53802.0,90145.0,-1.79,-19.95,41.76,13.47,138.0,-0.60,1.60,9.18,XY24,NTT,PAINTS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,99.79,M-SC,9.37,86074.0,-14703.0,14796.0,0.38,-14.59,17.19,0.09,245.0,-0.99,0.64,13.08,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,87.86,H-SC,16.23,118476.0,-24003.0,43528.0,0.46,-16.85,36.74,13.70,163.0,-0.55,0.88,39.83,OX40N,NTT,BANKS
43,INDIGOPNTS,1407.73,1408.0,116.52,M-SC,2.02,141806.0,-32753.0,32786.0,-1.52,-18.76,23.12,0.02,221.0,-1.00,1.05,22.97,OX40N,NTT,PAINTS
20,CERA,7989.07,9475.0,-18.14,H-SC,3.10,116685.0,-27118.0,53862.0,-0.27,-18.86,46.16,18.60,149.0,-0.50,0.87,28.77,OX40N,NTT,CERAMICS
50,KANSAINER,299.63,340.0,-68.70,H-SC,2.94,215865.0,-53802.0,90145.0,-1.79,-19.95,41.76,13.47,138.0,-0.60,1.60,9.18,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-26.76,H-LC,16.18,242455.0,-53479.0,147388.0,0.32,-18.07,60.79,31.73,1.0,-0.36,1.80,3.51,X40,BTT,IT
45,INFY,1461.46,2275.00,-18.16,H-LC,11.94,267963.0,6362.0,139260.0,0.05,2.43,51.97,55.67,3.0,0.05,1.99,8.69,X40,BTT,IT
47,ITC,411.90,452.00,-38.34,H-LC,3.06,147802.0,-2130.0,16731.0,-0.75,-1.42,11.32,9.74,4.0,-0.13,1.10,4.73,X40,NTT,FMCG
89,VBL,492.64,671.64,-8.60,H-LC,9.29,277635.0,12595.0,83707.0,-0.31,4.75,30.15,36.33,5.0,0.15,2.06,18.68,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.03,H-LC,11.94,200168.0,-16351.0,109252.0,0.22,-7.55,54.58,42.91,7.0,-0.15,1.48,4.83,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,12.80,H-MC,14.36,161159.0,34959.0,11217.0,0.82,27.70,6.96,36.59,108.0,3.12,1.2,43.69,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-8.01,H-LC,8.70,229776.0,45295.0,25069.0,-1.03,24.55,10.91,38.14,38.0,1.81,1.7,37.76,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,12.80,H-MC,14.36,161159.0,34959.0,11217.0,0.82,27.70,6.96,36.59,108.0,3.12,1.20,43.69,XY24,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2326.38,11.01,H-MC,5.63,191610.0,26940.0,41024.0,-0.59,16.36,21.41,41.28,89.0,0.66,1.42,25.65,X40N,ATH,AC
51,KPIGREEN,497.21,731.35,9.79,H-SC,11.16,130990.0,5693.0,53313.0,-0.34,4.54,40.70,47.09,141.0,0.11,0.97,63.94,MH,ATH,POWER
90,VOLTAS,1278.28,1918.49,-2.90,H-MC,7.37,203760.0,12018.0,84010.0,-0.83,6.27,41.23,50.08,99.0,0.14,1.51,13.49,XY25,ATH,AC
19,CAMS,3643.00,5211.76,-4.05,H-SC,5.44,109183.0,7179.0,36751.0,1.50,7.04,33.66,43.06,122.0,0.20,0.81,25.36,X40N,ATH,MISC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-4.90,H-MC,11.30,176606.0,25693.0,31612.0,-1.75,17.02,17.90,37.97,93.0,0.81,1.31,46.64,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-49.60,H-SC,12.09,140969.0,17633.0,116130.0,1.38,14.30,82.38,108.45,119.0,0.15,1.05,39.85,AR,ATH,MISC
61,RAJOOENG,102.50,143.33,-29.34,H-SC,8.73,107220.0,4720.0,36112.0,-0.60,4.60,33.68,39.83,114.0,0.13,0.80,6.16,AR,ATH,MISC
51,KPIGREEN,497.21,731.35,9.79,H-SC,11.16,130990.0,5693.0,53313.0,-0.34,4.54,40.70,47.09,141.0,0.11,0.97,63.94,MH,ATH,POWER
91,WHIRLPOOL,1219.98,2270.00,-41.67,M-SC,5.26,98498.0,7000.0,71756.0,1.17,7.65,72.85,86.07,223.0,0.10,0.73,42.49,XR,NTT,DURABLES


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,75.41,137.00,7155.56,L-SC,7.11,79616.0,-14194.0,90810.0,-0.53,-15.13,114.06,81.67,269.0,-0.16,0.59,54.18,XR,NTT,CERAMICS
23,COFFEEDAY,59.14,80.00,-39.64,L-SC,23.30,88397.0,-25152.0,65202.0,15.77,-22.15,73.76,35.27,268.0,-0.39,0.66,116.35,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-0.30,H-SC,15.73,92841.0,-8122.0,102654.0,0.86,-8.04,110.57,93.63,148.0,-0.08,0.69,38.29,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-18.25,H-SC,12.34,93105.0,-4875.0,26749.0,-1.18,-4.98,28.73,22.32,152.0,-0.18,0.69,35.03,XR,ATH,FINANCE
9,BAJAJFINSV,1994.80,2758.00,-9.31,H-LC,6.92,97167.0,-578.0,37973.0,1.25,-0.59,39.08,38.26,35.0,-0.02,0.72,27.23,X40,BTT,FINANCE


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-39.64,L-SC,23.30,88397.0,-25152.0,65202.0,15.77,-22.15,73.76,35.27,268.0,-0.39,0.66,116.35,XR,NTT,HOTELS
28,DMART,3764.92,5201.00,-8.01,H-LC,8.70,229776.0,45295.0,25069.0,-1.03,24.55,10.91,38.14,38.0,1.81,1.70,37.76,X40N,NTT,FMCG
35,HEROMOTOCO,4311.81,5949.07,-4.90,H-MC,11.30,176606.0,25693.0,31612.0,-1.75,17.02,17.90,37.97,93.0,0.81,1.31,46.64,AR,ATH,AUTO
4,APOLLOHOSP,7098.95,8285.00,-12.98,H-LC,7.30,173855.0,17678.0,8415.0,0.25,11.32,4.84,16.71,22.0,2.10,1.29,30.73,X40N,BTT,HEALTHCARE
5,ASIANPAINT,2961.56,4250.00,-10.45,H-LC,5.24,174780.0,-26606.0,114219.0,0.01,-13.21,65.35,43.51,27.0,-0.23,1.30,21.66,X40,BTT,PAINTS


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.79
1,25,41.36
2,50,70.95


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.54
LC    36.19
MC    26.29
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.46
X40      14.39
X40N     13.84
XY25     11.85
XR        9.10
AR        8.45
OX40N     7.45
X5K       2.23
MH        1.72
X200      1.40
SR        1.13
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.96
H-MC    23.39
H-SC    22.68
M-SC    13.24
M-LC     7.19
M-MC     2.58
L-SC     1.62
L-LC     1.04
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.57,1.03,29.28
IT,11.90,-16.40,74.54
FINANCE,8.06,-19.90,64.26
BANKS,7.60,-11.33,58.60
MISC,5.87,-16.33,83.01
PAINTS,5.55,-16.78,39.80
ELECTRICAL,4.73,-10.15,57.86
AUTO,4.55,-10.20,58.14
AC,3.66,1.94,31.27


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2801308.0,23
AR,1069776.0,10
XR,1067184.0,13
X40,797753.0,11
X40N,499541.0,11
OX40N,454999.0,10
XY25,448597.0,8
SR,191477.0,2
X5K,137766.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2717103.0,27
M-SC,1461962.0,17
H-MC,1226472.0,18
H-LC,1132573.0,19
M-LC,409347.0,5
M-MC,329802.0,2
L-SC,243176.0,3
L-MC,58855.0,1
L-LC,44018.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      920666.0      6
M-SC       XY24      860821.0      7
H-SC       AR        651865.0      5
           XR        514899.0      6
H-LC       X40       510623.0      6
H-MC       XY24      480290.0      5
M-MC       XY24      329802.0      2
H-LC       AR        228255.0      2
H-MC       X40       206274.0      4
H-SC       OX40N     197172.0      4
H-MC       XY25      194404.0      2
H-SC       SR        191477.0      2
H-LC       X40N      189768.0      5
H-SC       X40N      187711.0      3
M-LC       XY24      165146.0      2
M-SC       AR        158044.0      2
L-SC       XR        156012.0      2
M-LC       X5K       129631.0      1
H-MC       X40N      122062.0      3
M-SC       OX40N     121887.0      4
           XY25      121322.0      1
           XR        119032.0      2
H-MC       XR        113392.0      1
M-LC       XR        104994.0      1
L-SC       OX40N      87164.0      1
M-SC       X40        80856.0      1
H-LC       XY25       79277.0      3
           X200       71932.0      1
L-MC       XR         58855.0      1
H-SC       MH         53313.0      1
H-MC       OX40N      48776.0      1
H-LC       XY24       44583.0      1
L-LC       XY25       44018.0      1
H-MC       AR         31612.0      1
           MH         29662.0      1
M-LC       XY25        9576.0      1
H-LC       X5K         8135.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 54.0 seconds
